In [1]:
%pip install psycopg2

In [2]:
import os

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [3]:

import psycopg2

In [4]:
#find csv files in my current working directory and isolate them
import contextlib

csv_files = [file for file in os.listdir(os.getcwd()) if file.endswith("csv")]
csv_files

['Identification clients gps 2020.csv',
 'Identification clients gps 2021.csv',
 'Identification clients gps 2022.csv']

In [5]:
#make a new directory and move them There
dataset_dir = "GPS_datasets"
with contextlib.suppress(Exception):
    mkdir = f"mkdir {dataset_dir}"
    os.system(mkdir)



In [6]:
import shutil


In [7]:
with contextlib.suppress(Exception):
    for csv in csv_files:
        shutil.move(csv,dataset_dir)
       
        


Create pandas dataframe from csv file

In [8]:
data_path = f'{os.getcwd()}/{dataset_dir}/'
try:
    df = {file: pd.read_csv(data_path+file) for file in csv_files}
except UnicodeDecodeError:
    df = {file: pd.read_csv(data_path+file,encoding="ISO-8859-1") for file in csv_files}


Clean tables names and column names

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Date,N°,Noms,Contacts,Marque voiture,Immatriculation,Modèle GPS,N° GPS,IMEI GPS,Mot de passe GPS,Utilisateur cpte en ligne,Mot de passe cpte en ligne,Observation
3,NaN,2022-03-07 00:00:00,1,Sangaré Hamed,07 00 00 07 00,NaN,NaN,GPS Noir,07 00 51 13 11,864895035368024,123456,NaN,NaN,NaN
4,NaN,2022-04-05 00:00:00,2,Coulibaly Sehenan,07 07 87 11 87,SANDA Firstone,NaN,GPS-311 B,07 03 17 24 44,865011033212896,123456,NaN,NaN,NaN
5,NaN,2022-04-05 00:00:00,3,Coulibaly Sehenan,07 07 87 11 87,SANDA moto12,ZF210471,GPS-311 B,07 03 10 08 58,865011033212748,123456,NaN,NaN,NaN
6,NaN,2022-04-05 00:00:00,4,Coulibaly Sehenan,07 07 87 11 87,SANDA moto13,ZF210597,GPS-311 B,07 03 10 08 54,865011033214058,123456,NaN,NaN,NaN
7,NaN,2022-04-05 00:00:00,5,Coulibaly Sehenan,07 07 87 11 87,SANDA MOTO 1,ZF210576,GPS-311 B,07 03 20 60 66,NaN,123456,NaN,NaN,NaN
8,NaN,2022-04-05 00:00:00,6,Coulibaly Sehenan,07 07 87 11 87,SANDA MOTO 2,ZF210448,GPS-311 B,07 02 32 19 64,NaN,123456,NaN,NaN,NaN
9,NaN,2022-04-05 00:00:00,7,Coulibaly Sehenan,07 07 87 11 87,SANDA MOTO 3,ZF210408,GPS-311 B,07 02 02 56 46,NaN,123456,NaN,NaN,NaN


In [9]:
for k in csv_files:
    dataframe=df[k]
    clean_tbl_name=k.lower().replace(" ","_").replace("?","") \
        .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","_") \
            .replace(")","").replace(",","").replace("$","").replace(":","")
    #remove.csv extension
    tbl_name = f'{clean_tbl_name.split(".")[0]}'
    
    dataframe.columns = [x.lower().replace(" ", "_").replace("?", "")
                         .replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("%", "_") \
                             .replace("(","")
                         .replace(")", "").replace(",", "").replace("$", "") for x in dataframe.columns]

    #replacement dictionary that maps pandas dtypes to sql dtypes
    replacements={
        "object": "varchar",
        "float64": "float",
        "int64": "int",
        "datetime64": "timestamp",
        "timedelta64": "timestamp",
        "string": "varchar",

    }

    #table schema
    col_str = ", ".join(f"{n} {d}" for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    #adding the db connection
    host = "127.0.0.1"
    port = "5432"
    dbname = "Identification_clients_gps"
    user = "postgres"
    password = "Candycove456"
    conn_string = f"host={host} dbname={dbname} user={user} password={password}"
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    #drop table with same name
    cursor.execute(f"drop table if exists {tbl_name};")
  
    #create table
    cursor.execute("create table %s (%s);" % (tbl_name, col_str))
    #insert values to table
    dataframe.to_csv(k,header=dataframe.columns,index=False,encoding="utf-8")
    my_file=open(k)
    SQL_STATEMENT="""
    COPY %s FROM STDIN WITH 
    CSV 
    HEADER 
     DELIMITER AS ','
    """
    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name,file=my_file)
    cursor.execute("grant select on table %s to public" % tbl_name)
    conn.commit()
    conn.close()
    
